# Intro to Apache Spark

* [Intro to Spark slides](https://github.com/databricks/tech-talks/blob/master/2020-04-29%20%7C%20Intro%20to%20Apache%20Spark/Intro%20to%20Spark.pdf)
* What is a Spark DataFrame?
  * Read in the [NYT data set](https://github.com/nytimes/covid-19-data) 
* How to perform a distributed count?
* Transformations vs. Actions
* Spark SQL

[Spark docs](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)

In [2]:
%fs ls databricks-datasets/COVID/covid-19-data/

path name size dbfs:/databricks-datasets/COVID/covid-19-data/.git/ .git/ 0 dbfs:/databricks-datasets/COVID/covid-19-data/.github/ .github/ 0 dbfs:/databricks-datasets/COVID/covid-19-data/.gitignore .gitignore 10 dbfs:/databricks-datasets/COVID/covid-19-data/LICENSE LICENSE 1289 dbfs:/databricks-datasets/COVID/covid-19-data/NYT-readme.md NYT-readme.md 1748 dbfs:/databricks-datasets/COVID/covid-19-data/PROBABLE-CASES-NOTE.md PROBABLE-CASES-NOTE.md 3162 dbfs:/databricks-datasets/COVID/covid-19-data/README.md README.md 26125 dbfs:/databricks-datasets/COVID/covid-19-data/excess-deaths/ excess-deaths/ 0 dbfs:/databricks-datasets/COVID/covid-19-data/live/ live/ 0 dbfs:/databricks-datasets/COVID/covid-19-data/mask-use/ mask-use/ 0 dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv us-counties.csv 15996082 dbfs:/databricks-datasets/COVID/covid-19-data/us-states.csv us-states.csv 278655 dbfs:/databricks-datasets/COVID/covid-19-data/us.csv us.csv 4451

## How do we represent this data?

![Unified Engine](https://files.training.databricks.com/images/105/unified-engine.png)


####At first there were RDDs...
* **R**esilient: Fault-tolerant
* **D**istributed: Across multiple nodes
* **D**ataset: Collection of partitioned data

RDDs are immutable once created and keep track of their lineage to enable failure recovery.

####... and then there were DataFrames
* Higher-level APIs
* User friendly
* Optimizations and performance improvements

![RDD vs DataFrames](https://files.training.databricks.com/images/105/rdd-vs-dataframes.png)

###Create a DataFrame from the NYT COVID data

In [6]:
covid_df = spark.read.csv("dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv")
covid_df.show()

+----------+-----------+----------+-----+-----+------+
 _c0| _c1| _c2| _c3| _c4| _c5|
+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California|06059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona|04013| 1| 0|
2020-01-26|Los Angeles|California|06037| 1| 0|
2020-01-26| Orange|California|06059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona|04013| 1| 0|
2020-01-27|Los Angeles|California|06037| 1| 0|
2020-01-27| Orange|California|06059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona|04013| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

Let's look at the [Spark docs](https://spark.apache.org/docs/latest/index.html) to see what options we have to pass into the csv reader.

In [8]:
covid_df = spark.read.csv("dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True, inferSchema=True)
covid_df.show()

+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [9]:
covid_df.count()

Out[5]: 402068

### Let's write some Spark code!

* What about th information for the county where from 06.August, all COVID-19 tests are free (Los Angeles)
* I want the most recent information at the top

In [11]:
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles")) 

Out[3]: DataFrame[date: string, county: string, state: string, fips: int, cases: int, deaths: int]

**...nothing happened. Why?**

## Transformations vs Actions

There are two types of operations in Spark: transformations and actions.

Fundamental to Apache Spark are the notions that
* Transformations are **LAZY**
* Actions are **EAGER**

In [14]:
# same operations as above
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles")) 

Out[7]: DataFrame[date: string, county: string, state: string, fips: int, cases: int, deaths: int]

Why isn't is showing me results? **Sort** and **filter** are `transformations`, which are lazily evaluated in Spark.

Laziness has a number of benefits
* Not forced to load all data in the first step
  * Technically impossible with **REALLY** large datasets.
* Easier to parallelize operations 
  * N different transformations can be processed on a single data element, on a single thread, on a single machine. 
* Most importantly, it allows the framework to automatically apply various optimizations
  * This is also why we use Dataframes!
  
There's a lot Spark's **Catalyst** optimizer can do. Let's focus on only this situation. For more information, read [this blog!](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)
  
![Catalyst](https://files.training.databricks.com/images/105/catalyst-diagram.png)

In [16]:
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles") 
 .show())  #action!

+----------+-----------+----------+----+------+------+
 date| county| state|fips| cases|deaths|
+----------+-----------+----------+----+------+------+
2020-08-04|Los Angeles|California|6037|195614| 4758|
2020-08-03|Los Angeles|California|6037|193788| 4701|
2020-08-02|Los Angeles|California|6037|192167| 4692|
2020-08-01|Los Angeles|California|6037|190693| 4669|
2020-07-31|Los Angeles|California|6037|188481| 4621|
2020-07-30|Los Angeles|California|6037|185872| 4552|
2020-07-29|Los Angeles|California|6037|183383| 4516|
2020-07-28|Los Angeles|California|6037|178642| 4426|
2020-07-27|Los Angeles|California|6037|176028| 4375|
2020-07-26|Los Angeles|California|6037|173995| 4360|
2020-07-25|Los Angeles|California|6037|172325| 4351|
2020-07-24|Los Angeles|California|6037|168757| 4300|
2020-07-23|Los Angeles|California|6037|166848| 4262|
2020-07-22|Los Angeles|California|6037|164870| 4213|
2020-07-21|Los Angeles|California|6037|161673| 4154|
2020-07-20|Los Angeles|California|6037|159045| 4104|
2020-07-19|Los Angeles|California|6037|155887| 4095|
2020-07-18|Los Angeles|California|6037|153041| 4084|
2020-07-17|Los Angeles|California|6037|150319| 4047|
2020-07-16|Los Angeles|California|6037|147468| 3988|
+----------+-----------+----------+----+------+------+
only showing top 20 rows

###We can see the optimizations in action!
* Go to the Spark UI
* Click on the SQL query associated with your Spark job
* See the logical and physical plans!
  * The filter and sort have been swapped

## Now, Dive into Spark SQL

In [19]:
covid_df.createOrReplaceTempView("covid")

In [20]:
%sql

SELECT * 
FROM covid

-- keys = date, grouping = county, values = cases

date county state fips cases deaths 2020-01-21 Snohomish Washington 53061 1 0 2020-01-22 Snohomish Washington 53061 1 0 2020-01-23 Snohomish Washington 53061 1 0 2020-01-24 Cook Illinois 17031 1 0 2020-01-24 Snohomish Washington 53061 1 0 2020-01-25 Orange California 6059 1 0 2020-01-25 Cook Illinois 17031 1 0 2020-01-25 Snohomish Washington 53061 1 0 2020-01-26 Maricopa Arizona 4013 1 0 2020-01-26 Los Angeles California 6037 1 0 2020-01-26 Orange California 6059 1 0 2020-01-26 Cook Illinois 17031 1 0 2020-01-26 Snohomish Washington 53061 1 0 2020-01-27 Maricopa Arizona 4013 1 0 2020-01-27 Los Angeles California 6037 1 0 2020-01-27 Orange California 6059 1 0 2020-01-27 Cook Illinois 17031 1 0 2020-01-27 Snohomish Washington 53061 1 0 2020-01-28 Maricopa Arizona 4013 1 0 2020-01-28 Los Angeles California 6037 1 0 2020-01-28 Orange California 6059 1 0 2020-01-28 Cook Illinois 17031 1 0 2020-01-28 Snohomish Washington 53061 1 0 2020-01-29 Maricopa Arizona 4013 1 0 2020-01-29 Los Angeles California 6037 1 0 2020-01-29 Orange California 6059 1 0 2020-01-29 Cook Illinois 17031 1 0 2020-01-29 Snohomish Washington 53061 1 0 2020-01-30 Maricopa Arizona 4013 1 0 2020-01-30 Los Angeles California 6037 1 0 2020-01-30 Orange California 6059 1 0 2020-01-30 Cook Illinois 17031 2 0 2020-01-30 Snohomish Washington 53061 1 0 2020-01-31 Maricopa Arizona 4013 1 0 2020-01-31 Los Angeles California 6037 1 0 2020-01-31 Orange California 6059 1 0 2020-01-31 Santa Clara California 6085 1 0 2020-01-31 Cook Illinois 17031 2 0 2020-01-31 Snohomish Washington 53061 1 0 2020-02-01 Maricopa Arizona 4013 1 0 2020-02-01 Los Angeles California 6037 1 0 2020-02-01 Orange California 6059 1 0 2020-02-01 Santa Clara California 6085 1 0 2020-02-01 Cook Illinois 17031 2 0 2020-02-01 Suffolk Massachusetts 25025 1 0 2020-02-01 Snohomish Washington 53061 1 0 2020-02-02 Maricopa Arizona 4013 1 0 2020-02-02 Los Angeles California 6037 1 0 2020-02-02 Orange California 6059 1 0 2020-02-02 San Francisco California 6075 2 0 2020-02-02 Santa Clara California 6085 2 0 2020-02-02 Cook Illinois 17031 2 0 2020-02-02 Suffolk Massachusetts 25025 1 0 2020-02-02 Snohomish Washington 53061 1 0 2020-02-03 Maricopa Arizona 4013 1 0 2020-02-03 Los Angeles California 6037 1 0 2020-02-03 Orange California 6059 1 0 2020-02-03 San Francisco California 6075 2 0 2020-02-03 Santa Clara California 6085 2 0 2020-02-03 Cook Illinois 17031 2 0 2020-02-03 Suffolk Massachusetts 25025 1 0 2020-02-03 Snohomish Washington 53061 1 0 2020-02-04 Maricopa Arizona 4013 1 0 2020-02-04 Los Angeles California 6037 1 0 2020-02-04 Orange California 6059 1 0 2020-02-04 San Francisco California 6075 2 0 2020-02-04 Santa Clara California 6085 2 0 2020-02-04 Cook Illinois 17031 2 0 2020-02-04 Suffolk Massachusetts 25025 1 0 2020-02-04 Snohomish Washington 53061 1 0 2020-02-05 Maricopa Arizona 4013 1 0 2020-02-05 Los Angeles California 6037 1 0 2020-02-05 Orange California 6059 1 0 2020-02-05 San Francisco California 6075 2 0 2020-02-05 Santa Clara California 6085 2 0 2020-02-05 Cook Illinois 17031 2 0 2020-02-05 Suffolk Massachusetts 25025 1 0 2020-02-05 Snohomish Washington 53061 1 0 2020-02-05 Dane Wisconsin 55025 1 0 2020-02-06 Maricopa Arizona 4013 1 0 2020-02-06 Los Angeles California 6037 1 0 2020-02-06 Orange California 6059 1 0 2020-02-06 San Francisco California 6075 2 0 2020-02-06 Santa Clara California 6085 2 0 2020-02-06 Cook Illinois 17031 2 0 2020-02-06 Suffolk Massachusetts 25025 1 0 2020-02-06 Snohomish Washington 53061 1 0 2020-02-06 Dane Wisconsin 55025 1 0 2020-02-07 Maricopa Arizona 4013 1 0 2020-02-07 Los Angeles California 6037 1 0 2020-02-07 Orange California 6059 1 0 2020-02-07 San Francisco California 6075 2 0 2020-02-07 Santa Clara California 6085 2 0 2020-02-07 Cook Illinois 17031 2 0 2020-02-07 Suffolk Massachusetts 25025 1 0 2020-02-07 Snohomish Washington 53061 1 0 2020-02-07 Dane Wisconsin 55025 1 0 2020-02-08 Maricopa Arizona 4013 1 0 2020-02-08 Los Angeles California 6037 1 0 2020-02-

In [21]:
%sql

SELECT * 
FROM covid 
WHERE county = "Los Angeles"

-- keys = date, grouping = county, values = cases, deaths

date county state fips cases deaths 2020-01-26 Los Angeles California 6037 1 0 2020-01-27 Los Angeles California 6037 1 0 2020-01-28 Los Angeles California 6037 1 0 2020-01-29 Los Angeles California 6037 1 0 2020-01-30 Los Angeles California 6037 1 0 2020-01-31 Los Angeles California 6037 1 0 2020-02-01 Los Angeles California 6037 1 0 2020-02-02 Los Angeles California 6037 1 0 2020-02-03 Los Angeles California 6037 1 0 2020-02-04 Los Angeles California 6037 1 0 2020-02-05 Los Angeles California 6037 1 0 2020-02-06 Los Angeles California 6037 1 0 2020-02-07 Los Angeles California 6037 1 0 2020-02-08 Los Angeles California 6037 1 0 2020-02-09 Los Angeles California 6037 1 0 2020-02-10 Los Angeles California 6037 1 0 2020-02-11 Los Angeles California 6037 1 0 2020-02-12 Los Angeles California 6037 1 0 2020-02-13 Los Angeles California 6037 1 0 2020-02-14 Los Angeles California 6037 1 0 2020-02-15 Los Angeles California 6037 1 0 2020-02-16 Los Angeles California 6037 1 0 2020-02-17 Los Angeles California 6037 1 0 2020-02-18 Los Angeles California 6037 1 0 2020-02-19 Los Angeles California 6037 1 0 2020-02-20 Los Angeles California 6037 1 0 2020-02-21 Los Angeles California 6037 1 0 2020-02-22 Los Angeles California 6037 1 0 2020-02-23 Los Angeles California 6037 1 0 2020-02-24 Los Angeles California 6037 1 0 2020-02-25 Los Angeles California 6037 1 0 2020-02-26 Los Angeles California 6037 1 0 2020-02-27 Los Angeles California 6037 1 0 2020-02-28 Los Angeles California 6037 1 0 2020-02-29 Los Angeles California 6037 1 0 2020-03-01 Los Angeles California 6037 1 0 2020-03-02 Los Angeles California 6037 1 0 2020-03-03 Los Angeles California 6037 1 0 2020-03-04 Los Angeles California 6037 7 0 2020-03-05 Los Angeles California 6037 11 0 2020-03-06 Los Angeles California 6037 13 0 2020-03-07 Los Angeles California 6037 14 0 2020-03-08 Los Angeles California 6037 14 0 2020-03-09 Los Angeles California 6037 19 0 2020-03-10 Los Angeles California 6037 20 0 2020-03-11 Los Angeles California 6037 28 1 2020-03-12 Los Angeles California 6037 32 1 2020-03-13 Los Angeles California 6037 40 1 2020-03-14 Los Angeles California 6037 53 1 2020-03-15 Los Angeles California 6037 69 1 2020-03-16 Los Angeles California 6037 94 1 2020-03-17 Los Angeles California 6037 144 1 2020-03-18 Los Angeles California 6037 190 1 2020-03-19 Los Angeles California 6037 231 2 2020-03-20 Los Angeles California 6037 292 2 2020-03-21 Los Angeles California 6037 351 4 2020-03-22 Los Angeles California 6037 421 5 2020-03-23 Los Angeles California 6037 536 7 2020-03-24 Los Angeles California 6037 662 11 2020-03-25 Los Angeles California 6037 799 13 2020-03-26 Los Angeles California 6037 1216 21 2020-03-27 Los Angeles California 6037 1465 27 2020-03-28 Los Angeles California 6037 1804 33 2020-03-29 Los Angeles California 6037 2136 37 2020-03-30 Los Angeles California 6037 2474 44 2020-03-31 Los Angeles California 6037 3011 54 2020-04-01 Los Angeles California 6037 3518 65 2020-04-02 Los Angeles California 6037 4045 78 2020-04-03 Los Angeles California 6037 4566 89 2020-04-04 Los Angeles California 6037 5277 117 2020-04-05 Los Angeles California 6037 5940 132 2020-04-06 Los Angeles California 6037 6360 147 2020-04-07 Los Angeles California 6037 6910 169 2020-04-08 Los Angeles California 6037 7530 198 2020-04-09 Los Angeles California 6037 7955 223 2020-04-10 Los Angeles California 6037 8430 241 2020-04-11 Los Angeles California 6037 8873 265 2020-04-12 Los Angeles California 6037 9192 296 2020-04-13 Los Angeles California 6037 9420 320 2020-04-14 Los Angeles California 6037 10047 360 2020-04-15 Los Angeles California 6037 10496 402 2020-04-16 Los Angeles California 6037 10854 455 2020-04-17 Los Angeles California 6037 11391 495 2020-04-18 Los Angeles California 6037 12021 576 2020-04-19 Los Angeles California 6037 12341 600 2020-04-20 Los Angeles California 6037 13816 617 2020-04-21 Los Angeles California 6037 15140 663 2020-04-22 Los Angeles California 6037 16435 729 2020-04-

In [22]:
%sql

SELECT max(cases) AS max_cases, max(deaths) AS max_deaths, county 
FROM covid 
GROUP BY county 
ORDER BY max_cases DESC
LIMIT 10

max_cases max_deaths county 230964 23027 New York City 195614 4758 Los Angeles 124758 1724 Miami-Dade 121789 2153 Maricopa 107744 4902 Cook 79543 1367 Harris 58953 765 Broward 52131 722 Dallas 44997 716 Clark 43468 2044 Suffolk

###Get more data into the analysis

**This is census data taken from census.gov**
* It has enough information to be able to construct a fips code column that will correspond the the NYT data

In [25]:
%sh wget https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv && cp co-est2019-alldata.csv /dbfs/tmp

--2020-08-07 06:24:51-- https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv
Resolving www2.census.gov (www2.census.gov)... 23.198.24.18, 2a02:26f0:f4:286::208c, 2a02:26f0:f4:2b6::208c
Connecting to www2.census.gov (www2.census.gov)|23.198.24.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘co-est2019-alldata.csv’

 0K .......... .......... .......... .......... .......... 21.4M
 50K .......... .......... .......... .......... .......... 20.2M
 100K .......... .......... .......... .......... .......... 65.3M
 150K .......... .......... .......... .......... .......... 46.7M
 200K .......... .......... .......... .......... .......... 39.3M
 250K .......... .......... .......... .......... .......... 76.8M
 300K .......... .......... .......... .......... .......... 137M
 350K .......... .......... .......... .......... .......... 156M
 400K .......... .......... .......... .......... .......... 70.5M
 450K .......... .......... .......... .......... .......... 78.2M
 500K .......... .......... .......... .......... .......... 36.2M
 550K .......... .......... .......... .......... .......... 44.4M
 600K .......... .......... .......... .......... .......... 70.3M
 650K .......... .......... .......... .......... .......... 60.1M
 700K .......... .......... .......... .......... .......... 58.1M
 750K .......... .......... .......... .......... .......... 56.6M
 800K .......... .......... .......... .......... .......... 54.8M
 850K .......... .......... .......... .......... .......... 61.1M
 900K .......... .......... .......... .......... .......... 76.9M
 950K .......... .......... .......... .......... .......... 193M
 1000K .......... .......... .......... .......... .......... 187M
 1050K .......... .......... .......... .......... .......... 234M
 1100K .......... .......... .......... .......... .......... 219M
 1150K .......... .......... .......... .......... .......... 191M
 1200K .......... .......... .......... .......... .......... 211M
 1250K .......... .......... .......... .......... .......... 198M
 1300K .......... .......... .......... .......... .......... 218M
 1350K .......... .......... .......... .......... .......... 169M
 1400K .......... .......... .......... .......... .......... 4.20M
 1450K .......... .......... .......... .......... .......... 205M
 1500K .......... .......... .......... .......... .......... 20.5M
 1550K .......... .......... .......... .......... .......... 127M
 1600K .......... .......... .......... .......... .......... 6.20M
 1650K .......... .......... .......... .......... .......... 97.7M
 1700K .......... .......... .......... .......... .......... 78.5M
 1750K .......... .......... .......... .......... .......... 62.1M
 1800K .......... .......... .......... .......... .......... 53.8M
 1850K .......... .......... .......... .......... .......... 16.1M
 1900K .......... .......... .......... .......... .......... 129M
 1950K .......... .......... .......... .......... .......... 6.85M
 2000K .......... .......... .......... .......... .......... 81.9M
 2050K .......... .......... .......... .......... .......... 106M
 2100K .......... .......... .......... .......... .......... 11.8M
 2150K .......... .......... .......... .......... .......... 110M
 2200K .......... .......... .......... .......... .......... 66.8M
 2250K .......... .......... .......... .......... .......... 48.4M
 2300K .......... .......... .......... .......... .......... 89.0M
 2350K .......... .......... .......... .......... .......... 105M
 2400K .......... .......... .......... .......... .......... 96.2M
 2450K .......... .......... .......... .......... .......... 103M
 2500K .......... .......... .......... .......... .......... 52.5M
 2550K .......... .......... .......... .......... .......... 104M
 2600K .......... .......... .......... .......... .......... 85.0M
 2650

In [26]:
census_df = spark.read.csv("dbfs:/tmp/co-est2019-alldata.csv", header=True, inferSchema=True)

#display() is a Databricks only function. It displays the data, like show(), but also gives the visualization options we saw in the SQL section above
display(census_df)

SUMLEV REGION DIVISION STATE COUNTY STNAME CTYNAME CENSUS2010POP ESTIMATESBASE2010 POPESTIMATE2010 POPESTIMATE2011 POPESTIMATE2012 POPESTIMATE2013 POPESTIMATE2014 POPESTIMATE2015 POPESTIMATE2016 POPESTIMATE2017 POPESTIMATE2018 POPESTIMATE2019 NPOPCHG_2010 NPOPCHG_2011 NPOPCHG_2012 NPOPCHG_2013 NPOPCHG_2014 NPOPCHG_2015 NPOPCHG_2016 NPOPCHG_2017 NPOPCHG_2018 NPOPCHG_2019 BIRTHS2010 BIRTHS2011 BIRTHS2012 BIRTHS2013 BIRTHS2014 BIRTHS2015 BIRTHS2016 BIRTHS2017 BIRTHS2018 BIRTHS2019 DEATHS2010 DEATHS2011 DEATHS2012 DEATHS2013 DEATHS2014 DEATHS2015 DEATHS2016 DEATHS2017 DEATHS2018 DEATHS2019 NATURALINC2010 NATURALINC2011 NATURALINC2012 NATURALINC2013 NATURALINC2014 NATURALINC2015 NATURALINC2016 NATURALINC2017 NATURALINC2018 NATURALINC2019 INTERNATIONALMIG2010 INTERNATIONALMIG2011 INTERNATIONALMIG2012 INTERNATIONALMIG2013 INTERNATIONALMIG2014 INTERNATIONALMIG2015 INTERNATIONALMIG2016 INTERNATIONALMIG2017 INTERNATIONALMIG2018 INTERNATIONALMIG2019 DOMESTICMIG2010 DOMESTICMIG2011 DOMESTICMIG2012 DOMESTICMIG2013 DOMESTICMIG2014 DOMESTICMIG2015 DOMESTICMIG2016 DOMESTICMIG2017 DOMESTICMIG2018 DOMESTICMIG2019 NETMIG2010 NETMIG2011 NETMIG2012 NETMIG2013 NETMIG2014 NETMIG2015 NETMIG2016 NETMIG2017 NETMIG2018 NETMIG2019 RESIDUAL2010 RESIDUAL2011 RESIDUAL2012 RESIDUAL2013 RESIDUAL2014 RESIDUAL2015 RESIDUAL2016 RESIDUAL2017 RESIDUAL2018 RESIDUAL2019 GQESTIMATESBASE2010 GQESTIMATES2010 GQESTIMATES2011 GQESTIMATES2012 GQESTIMATES2013 GQESTIMATES2014 GQESTIMATES2015 GQESTIMATES2016 GQESTIMATES2017 GQESTIMATES2018 GQESTIMATES2019 RBIRTH2011 RBIRTH2012 RBIRTH2013 RBIRTH2014 RBIRTH2015 RBIRTH2016 RBIRTH2017 RBIRTH2018 RBIRTH2019 RDEATH2011 RDEATH2012 RDEATH2013 RDEATH2014 RDEATH2015 RDEATH2016 RDEATH2017 RDEATH2018 RDEATH2019 RNATURALINC2011 RNATURALINC2012 RNATURALINC2013 RNATURALINC2014 RNATURALINC2015 RNATURALINC2016 RNATURALINC2017 RNATURALINC2018 RNATURALINC2019 RINTERNATIONALMIG2011 RINTERNATIONALMIG2012 RINTERNATIONALMIG2013 RINTERNATIONALMIG2014 RINTERNATIONALMIG2015 RINTERNATIONALMIG2016 RINTERNATIONALMIG2017 RINTERNATIONALMIG2018 RINTERNATIONALMIG2019 RDOMESTICMIG2011 RDOMESTICMIG2012 RDOMESTICMIG2013 RDOMESTICMIG2014 RDOMESTICMIG2015 RDOMESTICMIG2016 RDOMESTICMIG2017 RDOMESTICMIG2018 RDOMESTICMIG2019 RNETMIG2011 RNETMIG2012 RNETMIG2013 RNETMIG2014 RNETMIG2015 RNETMIG2016 RNETMIG2017 RNETMIG2018 RNETMIG2019 40 3 6 1 0 Alabama Alabama 4779736 4780125 4785437 4799069 4815588 4830081 4841799 4852347 4863525 4874486 4887681 4903185 5312 13632 16519 14493 11718 10548 11178 10961 13195 15504 14226 59690 59067 57929 58903 59647 59389 58961 58271 57313 11075 48833 48366 50851 49712 51876 51710 53195 53665 53879 3151 10857 10701 7078 9191 7771 7679 5766 4606 3434 924 4665 5817 5046 3684 4580 5777 3011 3379 2772 1244 -1893 -114 2297 -959 -1544 -2157 2298 5279 9387 2168 2772 5703 7343 2725 3036 3620 5309 8658 12159 -7 3 115 72 -198 -259 -121 -114 -69 -89 116185 116246 115180 115793 116932 119032 119972 118619 117094 116576 116625 12.455519356 12.286865772 12.011401179 12.180258647 12.305777115 12.225150764 12.109454384 11.938128082 11.707442426 10.189987883 10.060889328 10.543799502 10.279697432 10.702541513 10.644438296 10.925228982 10.994485138 11.005972301 2.2655314734 2.2259764441 1.467601677 1.9005612146 1.6032356022 1.5807124672 1.1842254029 0.9436429432 0.7014701253 0.9734461014 1.2100275652 1.0462726847 0.7617960521 0.944900149 1.1891881655 0.6184014374 0.6922643302 0.5662420464 -0.395012534 -0.023713794 0.4762759328 -0.198306844 -0.318542758 -0.44401573 0.4719649629 1.0815221661 1.9175014754 0.5784335677 1.1863137707 1.5225486174 0.5634892079 0.6263573914 0.7451724354 1.0903664003 1.7737864964 2.4837435218 50 3 6 1 1 Alabama Autauga County 54571 54597 54773 55227 54954 54727 54893 54864 55243 55390 55533 55869 176 454 -273 -227 166 -29 379 147 143 336 150 638 615 571 640 651 666 676 631 624 157 514 560 582 573 584 547 573 518 541 -7 124 55 -11 67 67 119 103 113 83 25 4 -14 12 7 13 -3 -12 -7 -16 147 327 -329 -226 101 -107 266 59 37 270 172 331 

Let's tweak the DataFrame above to have a fips column that matches the NYT data. Here's the documentation on [user-defined functions (UDFs)](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html).

In [28]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def make_fips(state_code, county_code):
  if len(str(county_code)) == 1:
    return str(state_code) + "00" + str(county_code)
  elif len(str(county_code)) == 2:
    return str(state_code) + "0" + str(county_code)
  else:
    return str(state_code) + str(county_code)

make_fips_udf = udf(make_fips, StringType())
  
census_df = census_df.withColumn("fips", make_fips_udf(census_df.STATE, census_df.COUNTY))

Now that both the census and the covid data have an identical column, let's join the two DataFrames.

In [30]:
covid_with_census = (covid_df
                     .na.drop(subset=["fips"])
                     .join(census_df.drop("COUNTY", "STATE"), on=['fips'], how='inner'))

What do the cases look like for the most populous counties?

In [32]:
display(covid_with_census.filter("POPESTIMATE2019 > 2000000").select("county", "cases", "date"))

# keys = date, grouping = county, values = cases

county cases date Cook 1 2020-01-24 Orange 1 2020-01-25 Cook 1 2020-01-25 Maricopa 1 2020-01-26 Los Angeles 1 2020-01-26 Orange 1 2020-01-26 Cook 1 2020-01-26 Maricopa 1 2020-01-27 Los Angeles 1 2020-01-27 Orange 1 2020-01-27 Cook 1 2020-01-27 Maricopa 1 2020-01-28 Los Angeles 1 2020-01-28 Orange 1 2020-01-28 Cook 1 2020-01-28 Maricopa 1 2020-01-29 Los Angeles 1 2020-01-29 Orange 1 2020-01-29 Cook 1 2020-01-29 Maricopa 1 2020-01-30 Los Angeles 1 2020-01-30 Orange 1 2020-01-30 Cook 2 2020-01-30 Maricopa 1 2020-01-31 Los Angeles 1 2020-01-31 Orange 1 2020-01-31 Cook 2 2020-01-31 Maricopa 1 2020-02-01 Los Angeles 1 2020-02-01 Orange 1 2020-02-01 Cook 2 2020-02-01 Maricopa 1 2020-02-02 Los Angeles 1 2020-02-02 Orange 1 2020-02-02 Cook 2 2020-02-02 Maricopa 1 2020-02-03 Los Angeles 1 2020-02-03 Orange 1 2020-02-03 Cook 2 2020-02-03 Maricopa 1 2020-02-04 Los Angeles 1 2020-02-04 Orange 1 2020-02-04 Cook 2 2020-02-04 Maricopa 1 2020-02-05 Los Angeles 1 2020-02-05 Orange 1 2020-02-05 Cook 2 2020-02-05 Maricopa 1 2020-02-06 Los Angeles 1 2020-02-06 Orange 1 2020-02-06 Cook 2 2020-02-06 Maricopa 1 2020-02-07 Los Angeles 1 2020-02-07 Orange 1 2020-02-07 Cook 2 2020-02-07 Maricopa 1 2020-02-08 Los Angeles 1 2020-02-08 Orange 1 2020-02-08 Cook 2 2020-02-08 Maricopa 1 2020-02-09 Los Angeles 1 2020-02-09 Orange 1 2020-02-09 Cook 2 2020-02-09 Maricopa 1 2020-02-10 Los Angeles 1 2020-02-10 Orange 1 2020-02-10 San Diego 1 2020-02-10 Cook 2 2020-02-10 Maricopa 1 2020-02-11 Los Angeles 1 2020-02-11 Orange 1 2020-02-11 San Diego 1 2020-02-11 Cook 2 2020-02-11 Maricopa 1 2020-02-12 Los Angeles 1 2020-02-12 Orange 1 2020-02-12 San Diego 1 2020-02-12 Cook 2 2020-02-12 Bexar 1 2020-02-12 Maricopa 1 2020-02-13 Los Angeles 1 2020-02-13 Orange 1 2020-02-13 San Diego 1 2020-02-13 Cook 2 2020-02-13 Bexar 2 2020-02-13 Maricopa 1 2020-02-14 Los Angeles 1 2020-02-14 Orange 1 2020-02-14 San Diego 1 2020-02-14 Cook 2 2020-02-14 Bexar 2 2020-02-14 Maricopa 1 2020-02-15 Los Angeles 1 2020-02-15 Orange 1 2020-02-15 San Diego 1 2020-02-15 Cook 2 2020-02-15 Bexar 2 2020-02-15 Maricopa 1 2020-02-16 Los Angeles 1 2020-02-16 Orange 1 2020-02-16 San Diego 1 2020-02-16 Cook 2 2020-02-16 Bexar 2 2020-02-16 Maricopa 1 2020-02-17 Los Angeles 1 2020-02-17 Orange 1 2020-02-17 San Diego 1 2020-02-17 Cook 2 2020-02-17 Bexar 2 2020-02-17 Maricopa 1 2020-02-18 Los Angeles 1 2020-02-18 Orange 1 2020-02-18 San Diego 1 2020-02-18 Cook 2 2020-02-18 Bexar 2 2020-02-18 Maricopa 1 2020-02-19 Los Angeles 1 2020-02-19 Orange 1 2020-02-19 San Diego 1 2020-02-19 Cook 2 2020-02-19 Bexar 2 2020-02-19 Maricopa 1 2020-02-20 Los Angeles 1 2020-02-20 Orange 1 2020-02-20 San Diego 1 2020-02-20 Cook 2 2020-02-20 Bexar 2 2020-02-20 Maricopa 1 2020-02-21 Los Angeles 1 2020-02-21 Orange 1 2020-02-21 San Diego 1 2020-02-21 Cook 2 2020-02-21 Bexar 4 2020-02-21 Maricopa 1 2020-02-22 Los Angeles 1 2020-02-22 Orange 1 2020-02-22 San Diego 1 2020-02-22 Cook 2 2020-02-22 Bexar 4 2020-02-22 Maricopa 1 2020-02-23 Los Angeles 1 2020-02-23 Orange 1 2020-02-23 San Diego 1 2020-02-23 Cook 2 2020-02-23 Bexar 4 2020-02-23 Maricopa 1 2020-02-24 Los Angeles 1 2020-02-24 Orange 1 2020-02-24 San Diego 1 2020-02-24 Cook 2 2020-02-24 Bexar 10 2020-02-24 Maricopa 1 2020-02-25 Los Angeles 1 2020-02-25 Orange 1 2020-02-25 San Diego 1 2020-02-25 Cook 2 2020-02-25 Bexar 10 2020-02-25 Maricopa 1 2020-02-26 Los Angeles 1 2020-02-26 Orange 1 2020-02-26 San Diego 1 2020-02-26 Cook 2 2020-02-26 Bexar 10 2020-02-26 Maricopa 1 2020-02-27 Los Angeles 1 2020-02-27 Orange 1 2020-02-27 San Diego 1 2020-02-27 Cook 2 2020-02-27 Bexar 10 2020-02-27 Maricopa 1 2020-02-28 Los Angeles 1 2020-02-28 Orange 1 2020-02-28 San Diego 1 2020-02-28 Cook 2 2020-02-28 Bexar 11 2020-02-28 King 1 2020-02-28 Maricopa 1 2020-02-29 Los Angeles 1 2020-02-29 Orange 1 2020-02-29 San Diego 1 2020-02-29 Cook 3 2020-02-29 Bexar 11 2020-02-29 King 4 2020-02-29 Maricopa 1 2020-03-01 Los Angeles 1 2020-03-01 Orange 1 2020-03-01 San Diego 1 2020-03-01 Cook 3 2020-03-01

Since the NYT dataset has a new row for every day, with cases increasing each day, let's grab only the most recent numbers for each county.
* Below we're using the `col` function to refer to columns. It's equivalent to something like `df["column_name"]`
* To get the most recent row per county,  we'll use a [window function](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=window#pyspark.sql.Window)

In [34]:
from pyspark.sql.functions import row_number, col
from pyspark.sql import Window

w = Window.partitionBy("fips").orderBy(col("date").desc())
current_covid_rates = (covid_with_census
                       .withColumn("row_num", row_number().over(w))
                       .filter(col("row_num") == 1)
                       .drop("row_num"))

What counties are hardest hit when the cases are scaled with their population?

In [36]:
current_covid_rates = (current_covid_rates
                       .withColumn("case_rates_percent", 100*(col("cases")/col("POPESTIMATE2019")))
                       .sort(col("case_rates_percent").desc()))

#Look at the top 10 counties
display(current_covid_rates.select("county", "state", "cases", "POPESTIMATE2019", "case_rates_percent").limit(10))

county state cases POPESTIMATE2019 case_rates_percent Trousdale Tennessee 1576 11284 13.966678482807515 Lake Tennessee 756 7016 10.775370581527936 Lee Arkansas 890 8857 10.048549170147906 Dakota Nebraska 1904 20026 9.507640067911714 Lincoln Arkansas 1195 13024 9.17536855036855 Buena Vista Iowa 1786 19620 9.10295616717635 Nobles Minnesota 1749 21629 8.08636552776365 Bristol Bay Borough Alaska 66 836 7.894736842105263 East Carroll Louisiana 510 6861 7.433318758198514 Colfax Nebraska 695 10709 6.489868335045289